In [ ]:
import requests
import json

from ltr.client.solr_client import SolrClient

client = SolrClient()
host = client.get_host()

In [ ]:
# Rebuild the index if necessary
from ltr import download
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
download([corpus], dest='data/')
movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

## Reset LTR stores

The request below will clear any models under the test store.

In [ ]:
url = "http://{}:8983/solr/tmdb/schema/model-store/test".format(host)
print(url)
requests.delete(url)

This request will clear the test feature-store

In [ ]:
url = "http://{}:8983/solr/tmdb/schema/feature-store/test".format(host)
print(url)
requests.delete(url)

To upload a new feature-store named `test`:

In [ ]:
feature_set = [
    {
      "name" : "title_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "vote_average",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}vote_average"
      }
    }
   
]

url = 'http://{}:8983/solr/tmdb/schema/feature-store'.format(host)
print(url)
requests.put(url, json=feature_set)

To verify the new feature-store, issue the following GET request:

In [ ]:
url = 'http://{}:8983/solr/tmdb/schema/feature-store/test'.format(host)
print(url)
resp = requests.get(url)
resp.json()

## Searching and Logging

The following request will query and log features from the test feature-store.

In [ ]:
# grade    docId       query
# 4        7555        rambo
# ...
# 0        1368        rambo
# 4        12414       rocky

params = {
    "fl": "id,title,[features store=test efi.keywords=\"rambo\"]",
    'q': "id:7555 OR id:1370 id:1369 OR id:1368", #rambo's graded documents
    'rows': 10,
    'wt': 'json'  
}

url = 'http://{}:8983/solr/tmdb/select'.format(host)
print(url)
resp = requests.post(url, data=params).json()
resp

```
4    qid:1   1:12.31     2:10.56    3:6.4
4    qid:1   1:12.357876     2:11.95039    3:0.0
4    qid:1   1:7.010513     2:11.95039    3:0.0
3    qid:1   ...
...


...
4    qid:2


```

## Uploading a Model

Once features have been logged and training data has been generated, a model can be pushed into Solr. The following code shows how to upload a pre-trained model.

In [ ]:
model = """## LambdaMART
## No. of trees = 10
## No. of leaves = 10
## No. of threshold candidates = 256
## Learning rate = 0.1
## Stop early = 100

<ensemble>
	<tree id="1" weight="0.1">
		<split>
			<feature> 2 </feature>
			<threshold> 10.664251 </threshold>
			<split pos="left">
				<feature> 1 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> -1.8305741548538208 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 9.502127 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 7.0849166 </threshold>
						<split pos="left">
							<output> 0.23645669221878052 </output>
						</split>
						<split pos="right">
							<output> 1.7593677043914795 </output>
						</split>
					</split>
					<split pos="right">
						<output> 1.9719607830047607 </output>
					</split>
				</split>
			</split>
			<split pos="right">
				<feature> 2 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> 1.3728954792022705 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 8.602512 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 0.0 </threshold>
						<split pos="left">
							<feature> 2 </feature>
							<threshold> 13.815164 </threshold>
							<split pos="left">
								<output> 1.9401178359985352 </output>
							</split>
							<split pos="right">
								<output> 1.99532949924469 </output>
							</split>
						</split>
						<split pos="right">
							<feature> 1 </feature>
							<threshold> 11.085816 </threshold>
							<split pos="left">
								<output> 2.0 </output>
							</split>
							<split pos="right">
								<output> 1.99308180809021 </output>
							</split>
						</split>
					</split>
					<split pos="right">
						<output> 1.9870178699493408 </output>
					</split>
				</split>
			</split>
		</split>
	</tree>
</ensemble>
"""

from ltr.helpers.convert import convert

feature_mapping = [{'name': 'title_bm25'}, {'name': 'overview_bm25'}]

solr_model = convert(model, 'test', 'test', feature_mapping)
print(json.dumps(solr_model, indent=2))

In [ ]:
url = 'http://{}:8983/solr/tmdb/schema/model-store'.format(host)
print(url)
requests.put(url, json=solr_model)

## Searching with a Model
Now that a model is uploaded, we can use it to re-rank the results in Solr.

In [ ]:
params = {
    'q': '{!ltr model=test efi.keywords=\"rambo\"}',
    'rows': 10,
    'wt': 'json',
    'fl': 'title'
}

url = 'http://{}:8983/solr/tmdb/select'.format(host)
print(url)
resp = requests.post(url, data=params).json()
resp

## Rerank Query
The follwing example shows how to use a rerank query 

In [ ]:
params = {
    'q': 'rambo',
    'rows': 10,
    'wt': 'json',
    'fl': 'title',
    'rq': '{!ltr reRankDocs=50 model=test efi.keywords=\"rambo\"}'
}

url = 'http://{}:8983/solr/tmdb/select'.format(host)
print(url)
resp = requests.post(url, data=params).json()
resp